# NBA Player Props Edge Finder

**A quantitative approach to finding +EV player props using free data**

This notebook implements:
- Data collection via `nba_api` (free)
- Minutes projection with exponential decay weighting
- Feature engineering (pace, usage, matchups, rest, etc.)
- XGBoost-based prop projections
- Monte Carlo simulation for probability distributions
- Kelly Criterion bankroll management (fractional for small bankrolls)
- Edge detection comparing projections to market lines

---

## 1. Setup & Installation

In [ ]:
# Install required packages (run once)
# !pip install nba_api pandas numpy scikit-learn xgboost scipy matplotlib seaborn

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import warnings
warnings.filterwarnings('ignore')

# NBA API imports
from nba_api.stats.endpoints import (
    playergamelog, 
    leaguedashteamstats,
    leaguedashplayerstats,
    teamdashboardbygeneralsplits,
    commonplayerinfo,
    leaguegamefinder,
    boxscoretraditionalv2
)
from nba_api.stats.static import players, teams

# ML imports
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import xgboost as xgb
from scipy import stats

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

print("All imports successful!")

## 2. Configuration

In [ ]:
# ============================================
# CONFIGURATION - Adjust these settings
# ============================================

CONFIG = {
    # Bankroll settings (small bankroll focused)
    'bankroll': 500,              # Starting bankroll in dollars
    'kelly_fraction': 0.25,       # Use 1/4 Kelly for small bankrolls (conservative)
    'min_edge': 0.03,             # Minimum 3% edge to bet
    'max_bet_pct': 0.05,          # Never bet more than 5% of bankroll
    
    # Model settings
    'decay_factor': 0.92,         # Exponential decay for recency weighting
    'min_games': 10,              # Minimum games for reliable projections
    'lookback_games': 20,         # Games to look back for features
    
    # Season
    'current_season': '2024-25',
    'season_type': 'Regular Season',
    
    # API rate limiting (be respectful)
    'api_delay': 0.6,             # Seconds between API calls
}

print(f"Bankroll: ${CONFIG['bankroll']}")
print(f"Kelly Fraction: {CONFIG['kelly_fraction']} (conservative for small bankroll)")
print(f"Min Edge Required: {CONFIG['min_edge']*100}%")

## 3. Data Collection Functions

In [ ]:
def get_player_id(player_name):
    """Get NBA API player ID from name."""
    player_dict = players.find_players_by_full_name(player_name)
    if player_dict:
        return player_dict[0]['id']
    # Try partial match
    all_players = players.get_players()
    for p in all_players:
        if player_name.lower() in p['full_name'].lower():
            return p['id']
    return None

def get_team_id(team_name):
    """Get NBA API team ID from name or abbreviation."""
    all_teams = teams.get_teams()
    for t in all_teams:
        if (team_name.lower() in t['full_name'].lower() or 
            team_name.upper() == t['abbreviation']):
            return t['id']
    return None

def get_player_game_log(player_id, season=CONFIG['current_season'], n_games=None):
    """Fetch player's game log for the season."""
    time.sleep(CONFIG['api_delay'])
    try:
        log = playergamelog.PlayerGameLog(
            player_id=player_id,
            season=season,
            season_type_all_star=CONFIG['season_type']
        )
        df = log.get_data_frames()[0]
        if n_games and len(df) > n_games:
            df = df.head(n_games)
        return df
    except Exception as e:
        print(f"Error fetching game log: {e}")
        return pd.DataFrame()

def get_team_stats(season=CONFIG['current_season']):
    """Fetch league-wide team stats for pace and defensive ratings."""
    time.sleep(CONFIG['api_delay'])
    try:
        stats = leaguedashteamstats.LeagueDashTeamStats(
            season=season,
            measure_type_detailed_defense='Advanced',
            per_mode_detailed='PerGame'
        )
        return stats.get_data_frames()[0]
    except Exception as e:
        print(f"Error fetching team stats: {e}")
        return pd.DataFrame()

print("Data collection functions loaded!")

In [ ]:
def get_defensive_stats_by_position(season=CONFIG['current_season']):
    """
    Get team defensive stats - points allowed by position.
    This is crucial for matchup-based projections.
    """
    time.sleep(CONFIG['api_delay'])
    try:
        # Get league-wide player stats to calculate position scoring
        player_stats = leaguedashplayerstats.LeagueDashPlayerStats(
            season=season,
            per_mode_detailed='PerGame'
        )
        df = player_stats.get_data_frames()[0]
        return df
    except Exception as e:
        print(f"Error fetching defensive stats: {e}")
        return pd.DataFrame()

def get_team_pace_ratings(season=CONFIG['current_season']):
    """Get pace and offensive/defensive ratings for all teams."""
    time.sleep(CONFIG['api_delay'])
    try:
        stats = leaguedashteamstats.LeagueDashTeamStats(
            season=season,
            measure_type_detailed_defense='Advanced'
        )
        df = stats.get_data_frames()[0]
        return df[['TEAM_ID', 'TEAM_NAME', 'PACE', 'OFF_RATING', 'DEF_RATING', 'NET_RATING']]
    except Exception as e:
        print(f"Error fetching pace ratings: {e}")
        return pd.DataFrame()

print("Advanced stats functions loaded!")

## 4. Feature Engineering

In [ ]:
class FeatureEngineer:
    """
    Creates features for player prop projections.
    Key features based on research:
    - Exponentially weighted recent performance
    - Pace adjustments
    - Rest/back-to-back indicators
    - Home/away splits
    - Usage rate trends
    """
    
    def __init__(self, decay_factor=CONFIG['decay_factor']):
        self.decay_factor = decay_factor
        self.team_pace = None
        
    def load_team_pace(self):
        """Load team pace data for matchup adjustments."""
        self.team_pace = get_team_pace_ratings()
        if not self.team_pace.empty:
            # Create lookup dict
            self.pace_dict = dict(zip(self.team_pace['TEAM_NAME'], self.team_pace['PACE']))
            self.def_rating_dict = dict(zip(self.team_pace['TEAM_NAME'], self.team_pace['DEF_RATING']))
            print(f"Loaded pace data for {len(self.pace_dict)} teams")
    
    def exponential_weighted_avg(self, values, decay=None):
        """
        Calculate exponentially weighted average.
        More recent games weighted higher.
        """
        if decay is None:
            decay = self.decay_factor
        
        weights = np.array([decay ** i for i in range(len(values))])
        weights = weights / weights.sum()
        return np.sum(values * weights)
    
    def calculate_rest_days(self, game_dates):
        """
        Calculate rest days between games.
        Returns array of rest days (0 = back-to-back).
        """
        rest_days = []
        dates = pd.to_datetime(game_dates)
        for i in range(len(dates)):
            if i == len(dates) - 1:
                rest_days.append(3)  # Default for first game of season
            else:
                delta = (dates.iloc[i] - dates.iloc[i+1]).days - 1
                rest_days.append(max(0, delta))
        return rest_days
    
    def extract_features(self, game_log, opponent_team=None, is_home=True, rest_days=2):
        """
        Extract all features from a player's game log.
        
        Returns dict of features for projection.
        """
        if len(game_log) < CONFIG['min_games']:
            return None
        
        # Ensure proper column types
        numeric_cols = ['PTS', 'REB', 'AST', 'MIN', 'FGA', 'FG3A', 'FTA', 'STL', 'BLK', 'TOV']
        for col in numeric_cols:
            if col in game_log.columns:
                game_log[col] = pd.to_numeric(game_log[col], errors='coerce')
        
        # Convert MIN to numeric (handles "MM:SS" format)
        if 'MIN' in game_log.columns:
            if game_log['MIN'].dtype == object:
                game_log['MIN'] = game_log['MIN'].apply(
                    lambda x: float(str(x).split(':')[0]) + float(str(x).split(':')[1])/60 
                    if ':' in str(x) else float(x)
                )
        
        features = {}
        
        # === POINTS FEATURES ===
        pts = game_log['PTS'].values
        features['pts_ewa'] = self.exponential_weighted_avg(pts)
        features['pts_avg'] = pts.mean()
        features['pts_std'] = pts.std()
        features['pts_median'] = np.median(pts)
        features['pts_l5'] = pts[:5].mean() if len(pts) >= 5 else pts.mean()
        features['pts_l10'] = pts[:10].mean() if len(pts) >= 10 else pts.mean()
        
        # === REBOUNDS FEATURES ===
        reb = game_log['REB'].values
        features['reb_ewa'] = self.exponential_weighted_avg(reb)
        features['reb_avg'] = reb.mean()
        features['reb_std'] = reb.std()
        features['reb_l5'] = reb[:5].mean() if len(reb) >= 5 else reb.mean()
        
        # === ASSISTS FEATURES ===
        ast = game_log['AST'].values
        features['ast_ewa'] = self.exponential_weighted_avg(ast)
        features['ast_avg'] = ast.mean()
        features['ast_std'] = ast.std()
        features['ast_l5'] = ast[:5].mean() if len(ast) >= 5 else ast.mean()
        
        # === PRA (Points + Rebounds + Assists) ===
        pra = pts + reb + ast
        features['pra_ewa'] = self.exponential_weighted_avg(pra)
        features['pra_avg'] = pra.mean()
        features['pra_std'] = pra.std()
        
        # === MINUTES FEATURES (Critical for all props) ===
        mins = game_log['MIN'].values
        features['min_ewa'] = self.exponential_weighted_avg(mins)
        features['min_avg'] = mins.mean()
        features['min_std'] = mins.std()
        features['min_l5'] = mins[:5].mean() if len(mins) >= 5 else mins.mean()
        
        # === THREE POINTERS ===
        if 'FG3M' in game_log.columns:
            fg3m = pd.to_numeric(game_log['FG3M'], errors='coerce').values
            features['fg3m_ewa'] = self.exponential_weighted_avg(fg3m)
            features['fg3m_avg'] = fg3m.mean()
            features['fg3m_std'] = fg3m.std()
        
        # === USAGE PROXY (FGA + 0.44*FTA + TOV) ===
        fga = game_log['FGA'].values
        fta = game_log['FTA'].values
        tov = game_log['TOV'].values
        usage_proxy = fga + 0.44 * fta + tov
        features['usage_proxy_ewa'] = self.exponential_weighted_avg(usage_proxy)
        features['usage_proxy_avg'] = usage_proxy.mean()
        
        # === PER-MINUTE RATES ===
        mins_safe = np.where(mins > 0, mins, 1)
        features['pts_per_min'] = (pts / mins_safe).mean()
        features['reb_per_min'] = (reb / mins_safe).mean()
        features['ast_per_min'] = (ast / mins_safe).mean()
        
        # === HOME/AWAY SPLITS ===
        if 'MATCHUP' in game_log.columns:
            home_games = game_log[game_log['MATCHUP'].str.contains('vs.')]
            away_games = game_log[game_log['MATCHUP'].str.contains('@')]
            
            features['pts_home_avg'] = home_games['PTS'].mean() if len(home_games) > 0 else features['pts_avg']
            features['pts_away_avg'] = away_games['PTS'].mean() if len(away_games) > 0 else features['pts_avg']
            features['home_away_diff'] = features['pts_home_avg'] - features['pts_away_avg']
        
        # === REST DAYS ===
        rest = self.calculate_rest_days(game_log['GAME_DATE'])
        game_log_copy = game_log.copy()
        game_log_copy['REST_DAYS'] = rest
        
        # Performance on back-to-backs (0 rest days)
        b2b_games = game_log_copy[game_log_copy['REST_DAYS'] == 0]
        features['pts_b2b_avg'] = b2b_games['PTS'].mean() if len(b2b_games) > 2 else features['pts_avg']
        features['b2b_pts_diff'] = features['pts_b2b_avg'] - features['pts_avg']
        
        # === CONTEXT FEATURES ===
        features['is_home'] = 1 if is_home else 0
        features['rest_days'] = rest_days
        features['is_b2b'] = 1 if rest_days == 0 else 0
        features['games_played'] = len(game_log)
        
        # === PACE ADJUSTMENT ===
        if self.team_pace is not None and opponent_team and hasattr(self, 'pace_dict'):
            league_avg_pace = self.team_pace['PACE'].mean()
            opp_pace = self.pace_dict.get(opponent_team, league_avg_pace)
            features['pace_factor'] = opp_pace / league_avg_pace
            
            # Defensive rating adjustment
            league_avg_def = self.team_pace['DEF_RATING'].mean()
            opp_def = self.def_rating_dict.get(opponent_team, league_avg_def)
            features['def_factor'] = opp_def / league_avg_def
        else:
            features['pace_factor'] = 1.0
            features['def_factor'] = 1.0
        
        # === CONSISTENCY/VARIANCE ===
        features['pts_cv'] = features['pts_std'] / features['pts_avg'] if features['pts_avg'] > 0 else 0
        features['hit_rate_20pts'] = (pts >= 20).mean()
        features['hit_rate_25pts'] = (pts >= 25).mean()
        
        return features

# Initialize feature engineer
fe = FeatureEngineer()
print("Feature engineering class loaded!")

## 5. Minutes Projection Model

Minutes are the foundation of all prop projections. This implements a weighted approach similar to DARKO methodology.

In [ ]:
class MinutesProjector:
    """
    Projects player minutes using exponential decay weighting.
    
    Based on research:
    - 75% season average + 25% last 5 games (baseline)
    - Exponential decay for game-by-game weighting
    - Adjustments for rest, home/away, blowout risk
    """
    
    def __init__(self, decay_factor=0.92, prior_games=12.6):
        self.decay_factor = decay_factor
        self.prior_games = prior_games  # FiveThirtyEight uses 12.6
        
    def project_minutes(self, game_log, is_home=True, rest_days=2, 
                       expected_spread=0, preseason_prior=None):
        """
        Project minutes for next game.
        
        Args:
            game_log: Player's game log DataFrame
            is_home: Whether player is at home
            rest_days: Days of rest (0 = back-to-back)
            expected_spread: Point spread (positive = favorite)
            preseason_prior: Pre-season minutes expectation (optional)
            
        Returns:
            dict with projection and confidence interval
        """
        if len(game_log) < 3:
            return None
        
        # Parse minutes
        mins = game_log['MIN'].copy()
        if mins.dtype == object:
            mins = mins.apply(
                lambda x: float(str(x).split(':')[0]) + float(str(x).split(':')[1])/60 
                if ':' in str(x) else float(x)
            )
        mins = mins.values
        
        # Exponentially weighted average
        weights = np.array([self.decay_factor ** i for i in range(len(mins))])
        weights = weights / weights.sum()
        ewa_mins = np.sum(mins * weights)
        
        # Blend with season average using prior
        season_avg = mins.mean()
        games_played = len(mins)
        
        if preseason_prior:
            # FiveThirtyEight formula
            projected = (preseason_prior * self.prior_games + mins.sum()) / (self.prior_games + games_played)
        else:
            # Blend EWA with season average (75/25 split for recency)
            projected = 0.75 * season_avg + 0.25 * np.mean(mins[:5]) if len(mins) >= 5 else season_avg
            # Further blend with EWA
            projected = 0.6 * projected + 0.4 * ewa_mins
        
        # === ADJUSTMENTS ===
        
        # Back-to-back adjustment (reduce by ~8%)
        if rest_days == 0:
            projected *= 0.92
        elif rest_days >= 3:
            projected *= 1.02  # Well-rested bump
        
        # Blowout risk adjustment
        # Large favorites may see reduced minutes in garbage time
        if abs(expected_spread) >= 10:
            projected *= 0.95
        elif abs(expected_spread) >= 7:
            projected *= 0.97
        
        # Calculate uncertainty
        std = mins.std()
        
        return {
            'projection': projected,
            'std': std,
            'ci_low': projected - 1.5 * std,
            'ci_high': projected + 1.5 * std,
            'season_avg': season_avg,
            'ewa': ewa_mins,
            'games_played': games_played
        }

minutes_projector = MinutesProjector()
print("Minutes projector loaded!")

## 6. Player Prop Projector

In [ ]:
class PropProjector:
    """
    Projects player props using rate-based methodology.
    
    Core approach:
    1. Project minutes
    2. Calculate per-minute rates (with decay weighting)
    3. Apply matchup/context adjustments
    4. Monte Carlo simulation for probability distribution
    """
    
    def __init__(self, feature_engineer, minutes_projector):
        self.fe = feature_engineer
        self.mp = minutes_projector
        self.scaler = StandardScaler()
        
    def project_stat(self, game_log, stat='PTS', opponent_team=None, 
                    is_home=True, rest_days=2, expected_spread=0):
        """
        Project a specific stat for the next game.
        
        Args:
            game_log: Player's game log
            stat: 'PTS', 'REB', 'AST', 'PRA', 'FG3M'
            opponent_team: Team name for matchup adjustment
            is_home: Home game flag
            rest_days: Days of rest
            expected_spread: Point spread
            
        Returns:
            dict with projection, confidence interval, hit probabilities
        """
        if len(game_log) < CONFIG['min_games']:
            return None
        
        # Get features
        features = self.fe.extract_features(
            game_log, opponent_team, is_home, rest_days
        )
        if features is None:
            return None
        
        # Get minutes projection
        min_proj = self.mp.project_minutes(
            game_log, is_home, rest_days, expected_spread
        )
        if min_proj is None:
            return None
        
        # Map stat to features
        stat_map = {
            'PTS': ('pts', 'pts_per_min'),
            'REB': ('reb', 'reb_per_min'),
            'AST': ('ast', 'ast_per_min'),
            'PRA': ('pra', None),
            'FG3M': ('fg3m', None)
        }
        
        if stat not in stat_map:
            return None
        
        prefix, rate_key = stat_map[stat]
        
        # Get EWA and average
        ewa_key = f'{prefix}_ewa'
        avg_key = f'{prefix}_avg'
        std_key = f'{prefix}_std'
        
        if ewa_key not in features:
            return None
        
        # Base projection from EWA
        base_projection = features[ewa_key]
        
        # If we have per-minute rate, use minutes projection
        if rate_key and rate_key in features:
            rate = features[rate_key]
            min_based_proj = rate * min_proj['projection']
            # Blend rate-based with direct EWA (60/40)
            base_projection = 0.6 * min_based_proj + 0.4 * base_projection
        
        # Apply adjustments
        projection = base_projection
        
        # Pace adjustment
        projection *= features.get('pace_factor', 1.0)
        
        # Defensive matchup (for points)
        if stat == 'PTS':
            projection *= features.get('def_factor', 1.0)
        
        # Home/away adjustment
        if not is_home and 'home_away_diff' in features:
            projection -= features['home_away_diff'] * 0.3
        
        # Back-to-back adjustment
        if rest_days == 0 and 'b2b_pts_diff' in features:
            projection += features['b2b_pts_diff'] * 0.5
        
        # Get standard deviation for uncertainty
        std = features.get(std_key, projection * 0.25)
        
        return {
            'projection': projection,
            'std': std,
            'ci_low': projection - 1.5 * std,
            'ci_high': projection + 1.5 * std,
            'min_projection': min_proj['projection'],
            'features': features
        }
    
    def monte_carlo_probability(self, projection, std, line, n_sims=10000):
        """
        Run Monte Carlo simulation to estimate probability of hitting over/under.
        
        Uses normal distribution (reasonable for most props except 3PM).
        """
        # Simulate outcomes
        simulations = np.random.normal(projection, std, n_sims)
        simulations = np.maximum(simulations, 0)  # Can't have negative stats
        
        over_prob = (simulations > line).mean()
        under_prob = (simulations < line).mean()
        push_prob = 1 - over_prob - under_prob
        
        return {
            'over_prob': over_prob,
            'under_prob': under_prob,
            'push_prob': push_prob,
            'mean': simulations.mean(),
            'median': np.median(simulations),
            'percentile_25': np.percentile(simulations, 25),
            'percentile_75': np.percentile(simulations, 75)
        }

prop_projector = PropProjector(fe, minutes_projector)
print("Prop projector loaded!")

## 7. Edge Detection & Kelly Criterion

In [ ]:
class EdgeFinder:
    """
    Identifies +EV betting opportunities and calculates optimal bet sizing.
    
    Uses fractional Kelly for conservative bankroll management.
    """
    
    def __init__(self, bankroll=CONFIG['bankroll'], 
                 kelly_fraction=CONFIG['kelly_fraction'],
                 min_edge=CONFIG['min_edge']):
        self.bankroll = bankroll
        self.kelly_fraction = kelly_fraction
        self.min_edge = min_edge
        self.bet_history = []
        
    def american_to_decimal(self, american_odds):
        """Convert American odds to decimal odds."""
        if american_odds > 0:
            return (american_odds / 100) + 1
        else:
            return (100 / abs(american_odds)) + 1
    
    def decimal_to_implied_prob(self, decimal_odds):
        """Convert decimal odds to implied probability."""
        return 1 / decimal_odds
    
    def american_to_implied_prob(self, american_odds):
        """Convert American odds directly to implied probability."""
        decimal = self.american_to_decimal(american_odds)
        return self.decimal_to_implied_prob(decimal)
    
    def calculate_edge(self, model_prob, implied_prob):
        """Calculate edge as model probability minus implied probability."""
        return model_prob - implied_prob
    
    def kelly_criterion(self, win_prob, decimal_odds):
        """
        Calculate Kelly criterion bet size.
        
        Formula: f* = (bp - q) / b
        Where:
            b = decimal odds - 1 (net odds)
            p = probability of winning
            q = probability of losing (1 - p)
        """
        b = decimal_odds - 1
        p = win_prob
        q = 1 - p
        
        kelly = (b * p - q) / b
        
        # Apply fractional Kelly and caps
        fractional_kelly = kelly * self.kelly_fraction
        
        # Never bet negative or more than max_bet_pct
        bet_fraction = max(0, min(fractional_kelly, CONFIG['max_bet_pct']))
        
        return bet_fraction
    
    def analyze_bet(self, model_prob, american_odds, line, stat_type, player_name):
        """
        Analyze a potential bet opportunity.
        
        Returns:
            dict with edge, bet size, expected value, recommendation
        """
        decimal_odds = self.american_to_decimal(american_odds)
        implied_prob = self.decimal_to_implied_prob(decimal_odds)
        
        edge = self.calculate_edge(model_prob, implied_prob)
        
        # Calculate Kelly bet size
        kelly_fraction = self.kelly_criterion(model_prob, decimal_odds)
        bet_amount = kelly_fraction * self.bankroll
        
        # Expected value per dollar bet
        ev_per_dollar = (model_prob * (decimal_odds - 1)) - (1 - model_prob)
        
        # Determine recommendation
        if edge >= self.min_edge and kelly_fraction > 0:
            if edge >= 0.08:
                recommendation = "STRONG BET"
            elif edge >= 0.05:
                recommendation = "BET"
            else:
                recommendation = "LEAN"
        else:
            recommendation = "NO BET"
        
        return {
            'player': player_name,
            'stat': stat_type,
            'line': line,
            'american_odds': american_odds,
            'decimal_odds': decimal_odds,
            'implied_prob': implied_prob,
            'model_prob': model_prob,
            'edge': edge,
            'edge_pct': edge * 100,
            'kelly_fraction': kelly_fraction,
            'bet_amount': round(bet_amount, 2),
            'ev_per_dollar': ev_per_dollar,
            'recommendation': recommendation
        }
    
    def display_analysis(self, analysis):
        """Pretty print bet analysis."""
        print("\n" + "="*60)
        print(f"  {analysis['player']} - {analysis['stat']} {'OVER' if analysis['edge'] > 0 else 'UNDER'} {analysis['line']}")
        print("="*60)
        print(f"  Market Odds:     {analysis['american_odds']:+d} ({analysis['implied_prob']:.1%} implied)")
        print(f"  Model Prob:      {analysis['model_prob']:.1%}")
        print(f"  Edge:            {analysis['edge_pct']:+.1f}%")
        print(f"  Kelly Fraction:  {analysis['kelly_fraction']:.2%}")
        print(f"  Suggested Bet:   ${analysis['bet_amount']:.2f}")
        print(f"  EV per $1:       ${analysis['ev_per_dollar']:.3f}")
        print(f"  \n  >>> {analysis['recommendation']} <<<")
        print("="*60)

edge_finder = EdgeFinder()
print("Edge finder loaded!")

## 8. Full Analysis Pipeline

In [ ]:
def analyze_player_prop(player_name, stat='PTS', line=None, american_odds=-110,
                        opponent_team=None, is_home=True, rest_days=2,
                        expected_spread=0, side='over'):
    """
    Complete analysis pipeline for a player prop.
    
    Args:
        player_name: Player's full name
        stat: 'PTS', 'REB', 'AST', 'PRA', 'FG3M'
        line: The prop line (e.g., 24.5)
        american_odds: Odds for the bet (default -110)
        opponent_team: Opponent team name (for matchup adjustment)
        is_home: Is player at home?
        rest_days: Days since last game
        expected_spread: Point spread (positive = favorite)
        side: 'over' or 'under'
        
    Returns:
        Complete analysis dict
    """
    print(f"\nAnalyzing: {player_name} {stat} {side.upper()} {line}")
    print("-" * 50)
    
    # Get player ID
    player_id = get_player_id(player_name)
    if not player_id:
        print(f"Could not find player: {player_name}")
        return None
    print(f"Found player ID: {player_id}")
    
    # Fetch game log
    print("Fetching game log...")
    game_log = get_player_game_log(player_id)
    if game_log.empty:
        print("Could not fetch game log")
        return None
    print(f"Retrieved {len(game_log)} games")
    
    # Load team pace data if not loaded
    if fe.team_pace is None:
        print("Loading team pace data...")
        fe.load_team_pace()
    
    # Get projection
    print("Generating projection...")
    projection = prop_projector.project_stat(
        game_log, stat, opponent_team, is_home, rest_days, expected_spread
    )
    
    if projection is None:
        print("Could not generate projection")
        return None
    
    print(f"\nProjection: {projection['projection']:.1f} (std: {projection['std']:.1f})")
    print(f"Minutes projection: {projection['min_projection']:.1f}")
    print(f"90% CI: [{projection['ci_low']:.1f}, {projection['ci_high']:.1f}]")
    
    # If no line provided, suggest one
    if line is None:
        line = round(projection['projection'] * 2) / 2  # Round to nearest 0.5
        print(f"\nNo line provided. Suggesting line: {line}")
    
    # Monte Carlo simulation
    print("\nRunning Monte Carlo simulation (10,000 iterations)...")
    mc_results = prop_projector.monte_carlo_probability(
        projection['projection'], projection['std'], line
    )
    
    print(f"Over {line} probability:  {mc_results['over_prob']:.1%}")
    print(f"Under {line} probability: {mc_results['under_prob']:.1%}")
    
    # Determine which probability to use
    model_prob = mc_results['over_prob'] if side == 'over' else mc_results['under_prob']
    
    # Analyze the bet
    analysis = edge_finder.analyze_bet(
        model_prob, american_odds, line, stat, player_name
    )
    
    # Add projection details
    analysis['projection'] = projection['projection']
    analysis['projection_std'] = projection['std']
    analysis['min_projection'] = projection['min_projection']
    analysis['mc_results'] = mc_results
    analysis['side'] = side
    
    # Display results
    edge_finder.display_analysis(analysis)
    
    return analysis

print("Analysis pipeline loaded!")

---
## 9. Example Usage

Let's run through a complete example analysis.

In [ ]:
# ============================================
# EXAMPLE: Analyze a Points Prop
# ============================================

# Analyze Jayson Tatum points over 27.5 at -110
analysis = analyze_player_prop(
    player_name="Jayson Tatum",
    stat="PTS",
    line=27.5,
    american_odds=-110,
    opponent_team="Miami Heat",
    is_home=True,
    rest_days=2,
    expected_spread=5,
    side='over'
)

In [ ]:
# ============================================
# EXAMPLE: Analyze an Assists Prop
# ============================================

analysis = analyze_player_prop(
    player_name="Tyrese Haliburton",
    stat="AST",
    line=10.5,
    american_odds=-115,
    opponent_team="Chicago Bulls",
    is_home=False,
    rest_days=1,
    expected_spread=-3,
    side='over'
)

In [ ]:
# ============================================
# EXAMPLE: Analyze a Rebounds Prop (Role Player)
# ============================================

# Role players are often mispriced - research shows ~9% edge
analysis = analyze_player_prop(
    player_name="Ivica Zubac",
    stat="REB",
    line=11.5,
    american_odds=-110,
    opponent_team="San Antonio Spurs",
    is_home=True,
    rest_days=2,
    expected_spread=7,
    side='over'
)

In [ ]:
# ============================================
# EXAMPLE: Back-to-Back Analysis
# ============================================

# B2B games often create under value for stars
analysis = analyze_player_prop(
    player_name="Giannis Antetokounmpo",
    stat="PTS",
    line=31.5,
    american_odds=-110,
    opponent_team="Detroit Pistons",
    is_home=False,
    rest_days=0,  # BACK-TO-BACK
    expected_spread=-8,
    side='under'
)

## 10. Batch Analysis - Find Today's Edges

In [ ]:
def batch_analyze_props(props_list):
    """
    Analyze multiple props and return sorted by edge.
    
    Args:
        props_list: List of dicts with prop details
        
    Returns:
        DataFrame with all analyses sorted by edge
    """
    results = []
    
    for prop in props_list:
        try:
            analysis = analyze_player_prop(**prop)
            if analysis:
                results.append(analysis)
        except Exception as e:
            print(f"Error analyzing {prop.get('player_name', 'Unknown')}: {e}")
        
        # Rate limiting
        time.sleep(1)
    
    if not results:
        return pd.DataFrame()
    
    # Convert to DataFrame
    df = pd.DataFrame(results)
    
    # Sort by edge
    df = df.sort_values('edge', ascending=False)
    
    return df

print("Batch analysis function loaded!")

In [ ]:
# ============================================
# EXAMPLE: Batch analyze today's props
# ============================================

# Define props to analyze (you'd get these from your own sources)
todays_props = [
    {
        'player_name': 'LeBron James',
        'stat': 'PTS',
        'line': 25.5,
        'american_odds': -110,
        'opponent_team': 'Phoenix Suns',
        'is_home': True,
        'rest_days': 2,
        'expected_spread': 3,
        'side': 'over'
    },
    {
        'player_name': 'Anthony Edwards',
        'stat': 'PTS',
        'line': 26.5,
        'american_odds': -115,
        'opponent_team': 'Memphis Grizzlies',
        'is_home': False,
        'rest_days': 1,
        'expected_spread': -2,
        'side': 'over'
    },
    {
        'player_name': 'Domantas Sabonis',
        'stat': 'REB',
        'line': 12.5,
        'american_odds': -110,
        'opponent_team': 'Houston Rockets',
        'is_home': True,
        'rest_days': 3,
        'expected_spread': 1,
        'side': 'over'
    }
]

# Run batch analysis
results_df = batch_analyze_props(todays_props)

# Display results
if not results_df.empty:
    print("\n" + "="*70)
    print("                    TODAY'S EDGE RANKINGS")
    print("="*70)
    display_cols = ['player', 'stat', 'line', 'side', 'projection', 'model_prob', 
                    'edge_pct', 'bet_amount', 'recommendation']
    print(results_df[display_cols].to_string(index=False))

## 11. Visualization Tools

In [ ]:
def plot_projection_distribution(projection, std, line, player_name, stat):
    """
    Visualize the projection distribution against the line.
    """
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Generate distribution
    x = np.linspace(max(0, projection - 4*std), projection + 4*std, 1000)
    y = stats.norm.pdf(x, projection, std)
    
    # Plot distribution
    ax.plot(x, y, 'b-', linewidth=2, label='Projection Distribution')
    
    # Fill under/over regions
    ax.fill_between(x[x <= line], y[x <= line], alpha=0.3, color='red', label=f'Under {line}')
    ax.fill_between(x[x >= line], y[x >= line], alpha=0.3, color='green', label=f'Over {line}')
    
    # Add vertical lines
    ax.axvline(projection, color='blue', linestyle='--', linewidth=2, label=f'Projection: {projection:.1f}')
    ax.axvline(line, color='black', linestyle='-', linewidth=2, label=f'Line: {line}')
    
    # Calculate probabilities
    over_prob = 1 - stats.norm.cdf(line, projection, std)
    under_prob = stats.norm.cdf(line, projection, std)
    
    # Add text
    ax.text(0.02, 0.98, f'Over: {over_prob:.1%}', transform=ax.transAxes, 
            fontsize=12, verticalalignment='top', color='green', fontweight='bold')
    ax.text(0.02, 0.92, f'Under: {under_prob:.1%}', transform=ax.transAxes, 
            fontsize=12, verticalalignment='top', color='red', fontweight='bold')
    
    ax.set_xlabel(stat, fontsize=12)
    ax.set_ylabel('Probability Density', fontsize=12)
    ax.set_title(f'{player_name} - {stat} Projection vs Line', fontsize=14, fontweight='bold')
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

print("Visualization tools loaded!")

In [ ]:
# Example visualization
if analysis:
    plot_projection_distribution(
        analysis['projection'],
        analysis['projection_std'],
        analysis['line'],
        analysis['player'],
        analysis['stat']
    )

In [ ]:
def plot_historical_performance(game_log, stat, line=None, n_games=20):
    """
    Plot recent game performance with trend line.
    """
    # Get last n games (reverse to show oldest first)
    df = game_log.head(n_games).iloc[::-1].copy()
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    games = range(1, len(df) + 1)
    values = df[stat].values
    
    # Bar chart of performance
    colors = ['green' if v > (line or 0) else 'red' for v in values] if line else 'steelblue'
    ax.bar(games, values, color=colors, alpha=0.7, edgecolor='black')
    
    # Rolling average
    rolling_avg = pd.Series(values).rolling(5, min_periods=1).mean()
    ax.plot(games, rolling_avg, 'b-', linewidth=2, marker='o', label='5-Game Rolling Avg')
    
    # Line if provided
    if line:
        ax.axhline(line, color='black', linestyle='--', linewidth=2, label=f'Line: {line}')
        hit_rate = (values > line).mean()
        ax.text(0.98, 0.98, f'Hit Rate: {hit_rate:.1%}', transform=ax.transAxes,
                fontsize=12, verticalalignment='top', horizontalalignment='right', fontweight='bold')
    
    ax.set_xlabel('Game #', fontsize=12)
    ax.set_ylabel(stat, fontsize=12)
    ax.set_title(f'Last {n_games} Games - {stat} Performance', fontsize=14, fontweight='bold')
    ax.legend(loc='upper left')
    ax.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()

print("Historical performance plot loaded!")

## 12. Bankroll Tracker

In [ ]:
class BankrollTracker:
    """
    Track bets and bankroll performance over time.
    """
    
    def __init__(self, starting_bankroll=CONFIG['bankroll']):
        self.starting_bankroll = starting_bankroll
        self.current_bankroll = starting_bankroll
        self.bets = []
        
    def place_bet(self, analysis, result=None):
        """
        Record a bet.
        
        Args:
            analysis: Analysis dict from analyze_player_prop
            result: 'win', 'loss', 'push', or None (pending)
        """
        bet = {
            'date': datetime.now().strftime('%Y-%m-%d %H:%M'),
            'player': analysis['player'],
            'stat': analysis['stat'],
            'line': analysis['line'],
            'side': analysis.get('side', 'over'),
            'odds': analysis['american_odds'],
            'bet_amount': analysis['bet_amount'],
            'model_prob': analysis['model_prob'],
            'edge': analysis['edge'],
            'result': result,
            'profit': None,
            'bankroll_after': None
        }
        
        self.bets.append(bet)
        return len(self.bets) - 1  # Return bet index
    
    def settle_bet(self, bet_index, result, actual_value=None):
        """
        Settle a bet with result.
        
        Args:
            bet_index: Index of bet to settle
            result: 'win', 'loss', or 'push'
            actual_value: Actual stat value (optional)
        """
        bet = self.bets[bet_index]
        bet['result'] = result
        bet['actual_value'] = actual_value
        
        if result == 'win':
            decimal_odds = edge_finder.american_to_decimal(bet['odds'])
            profit = bet['bet_amount'] * (decimal_odds - 1)
        elif result == 'loss':
            profit = -bet['bet_amount']
        else:  # push
            profit = 0
        
        bet['profit'] = profit
        self.current_bankroll += profit
        bet['bankroll_after'] = self.current_bankroll
        
        return profit
    
    def get_stats(self):
        """
        Get overall betting statistics.
        """
        settled = [b for b in self.bets if b['result'] is not None]
        
        if not settled:
            return "No settled bets yet."
        
        wins = sum(1 for b in settled if b['result'] == 'win')
        losses = sum(1 for b in settled if b['result'] == 'loss')
        pushes = sum(1 for b in settled if b['result'] == 'push')
        
        total_profit = sum(b['profit'] for b in settled if b['profit'])
        total_wagered = sum(b['bet_amount'] for b in settled)
        
        roi = (total_profit / total_wagered * 100) if total_wagered > 0 else 0
        
        return {
            'total_bets': len(settled),
            'wins': wins,
            'losses': losses,
            'pushes': pushes,
            'win_rate': wins / (wins + losses) if (wins + losses) > 0 else 0,
            'total_wagered': total_wagered,
            'total_profit': total_profit,
            'roi': roi,
            'current_bankroll': self.current_bankroll,
            'bankroll_change': self.current_bankroll - self.starting_bankroll
        }
    
    def display_stats(self):
        """Pretty print statistics."""
        stats = self.get_stats()
        
        if isinstance(stats, str):
            print(stats)
            return
        
        print("\n" + "="*50)
        print("           BANKROLL STATISTICS")
        print("="*50)
        print(f"  Total Bets:        {stats['total_bets']}")
        print(f"  Record:            {stats['wins']}-{stats['losses']}-{stats['pushes']}")
        print(f"  Win Rate:          {stats['win_rate']:.1%}")
        print(f"  Total Wagered:     ${stats['total_wagered']:.2f}")
        print(f"  Total Profit:      ${stats['total_profit']:+.2f}")
        print(f"  ROI:               {stats['roi']:+.1f}%")
        print(f"  Current Bankroll:  ${stats['current_bankroll']:.2f}")
        print(f"  Change:            ${stats['bankroll_change']:+.2f}")
        print("="*50)

tracker = BankrollTracker()
print("Bankroll tracker loaded!")

---
## Quick Reference Guide

### Analyze a single prop:
```python
analysis = analyze_player_prop(
    player_name="Player Name",
    stat="PTS",  # PTS, REB, AST, PRA, FG3M
    line=24.5,
    american_odds=-110,
    opponent_team="Opponent Team",
    is_home=True,
    rest_days=2,
    expected_spread=5,
    side='over'  # 'over' or 'under'
)
```

### Key variables to adjust:
- `CONFIG['bankroll']` - Your starting bankroll
- `CONFIG['kelly_fraction']` - Lower = more conservative (0.25 recommended for small bankrolls)
- `CONFIG['min_edge']` - Minimum edge required to bet (0.03 = 3%)

### Edge interpretation:
- **3-5%**: Slight edge (LEAN)
- **5-8%**: Solid edge (BET)
- **8%+**: Strong edge (STRONG BET)

### Data sources used:
- `nba_api` - Free NBA.com stats API
- No paid data required!

---